In [40]:
#advanced/estimated stats
from nba_api.stats.endpoints import playercareerstats
from sqlalchemy import create_engine, text
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import playerestimatedmetrics
from nba_api.stats.endpoints import playerdashboardbyyearoveryear
import pandas as pd

metrics = playerestimatedmetrics.PlayerEstimatedMetrics(
    season='2023-24'
)
metrics_df = metrics.get_data_frames()[0]
metrics_df.set_index('PLAYER_ID')

advanced_stats = leaguedashplayerstats.LeagueDashPlayerStats(
    measure_type_detailed_defense='Advanced',
    per_mode_detailed='PerGame',
    season='2024-25',
    season_type_all_star='Regular Season'
)
advanced_df = advanced_stats.get_data_frames()[0]
advanced_df.set_index('PLAYER_ID')

print(metrics_df[metrics_df['PLAYER_ID'] == 2544]['E_OFF_RATING'].values[0])
print(metrics_df[metrics_df['PLAYER_ID'] == 2544]['E_DEF_RATING'].values[0])
print(metrics_df[metrics_df['PLAYER_ID'] == 2544]['E_NET_RATING'].values[0])

pd.set_option('display.max_columns', 78)
advanced_df

116.1
112.3
3.8


,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,E_OFF_RATING,OFF_RATING,sp_work_OFF_RATING,E_DEF_RATING,DEF_RATING,sp_work_DEF_RATING,E_NET_RATING,NET_RATING,sp_work_NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,E_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM,FGA,FGM_PG,FGA_PG,FG_PCT,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,E_OFF_RATING_RANK,OFF_RATING_RANK,sp_work_OFF_RATING_RANK,E_DEF_RATING_RANK,DEF_RATING_RANK,sp_work_DEF_RATING_RANK,E_NET_RATING_RANK,NET_RATING_RANK,sp_work_NET_RATING_RANK,AST_PCT_RANK,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,E_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,USG_PCT_RANK,E_USG_PCT_RANK,E_PACE_RANK,PACE_RANK,sp_work_PACE_RANK,PIE_RANK,FGM_RANK,FGA_RANK,FGM_PG_RANK,FGA_PG_RANK,FG_PCT_RANK
0,1630639,A.J. Lawson,A.J.,1610612761,TOR,24.0,6,3,3,0.500,5.1,119.9,115.9,115.9,117.1,109.8,109.8,2.9,6.0,6.0,0.000,0.00,0.0,0.032,0.115,0.070,8.3,8.1,0.500,0.486,0.164,0.176,92.86,97.48,81.23,97.48,0.031,63,4,10,0.7,1.7,0.400,495,480,54,267,486,38,90,90,504,238,238,168,99,99,525,513,525,325,281,330,160,153,361,453,289,267,538,516,516,513,486,493,474,471,420
1,1631260,AJ Green,AJ,1610612749,MIL,25.0,52,31,21,0.596,22.6,111.4,112.0,112.0,108.0,108.8,108.8,3.3,3.2,3.2,0.078,2.52,16.3,0.011,0.083,0.049,6.5,6.5,0.607,0.619,0.128,0.132,101.46,100.80,84.00,100.80,0.058,2470,134,314,2.6,6.0,0.427,137,77,318,152,215,207,196,196,177,191,191,159,165,165,411,99,302,495,448,489,72,71,73,96,453,459,279,321,321,454,226,215,276,256,346
2,1642358,AJ Johnson,AJ,1610612764,WAS,20.0,11,4,7,0.364,6.7,106.2,107.2,107.2,97.3,95.7,95.7,8.8,11.5,11.5,0.261,2.40,26.1,0.000,0.097,0.053,10.9,10.8,0.500,0.512,0.178,0.189,106.63,106.95,89.13,106.95,0.093,166,12,28,1.1,2.5,0.429,461,465,111,402,461,376,355,355,33,33,33,66,42,42,61,115,80,514,365,456,344,341,361,417,238,219,34,38,38,238,450,452,437,429,340
3,203932,Aaron Gordon,Aaron,1610612743,DEN,29.0,36,24,12,0.667,27.0,120.4,122.2,122.2,112.4,112.5,112.5,8.0,9.7,9.7,0.143,2.51,21.2,0.061,0.114,0.089,8.5,8.5,0.578,0.619,0.176,0.179,103.18,102.41,85.34,102.41,0.098,2075,167,327,4.6,9.1,0.511,302,183,171,91,139,36,18,18,356,334,334,76,56,56,228,101,165,153,287,219,178,177,128,94,245,252,139,182,182,204,185,208,130,141,116
4,1628988,Aaron Holiday,Aaron,1610612745,HOU,28.0,45,26,19,0.578,12.7,105.8,106.4,106.4,102.9,101.6,101.6,2.9,4.7,4.7,0.137,2.07,20.2,0.015,0.070,0.043,9.7,9.7,0.541,0.571,0.154,0.160,102.66,103.05,85.87,103.05,0.076,1211,77,182,1.7,4.0,0.423,216,145,278,174,370,388,377,377,59,61,61,166,125,125,246,173,189,461,497,509,272,272,234,228,336,335,167,154,154,347,311,298,362,351,359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,203897,Zach LaVine,Zach,1610612758,SAC,29.0,53,25,28,0.472,34.4,115.1,115.9,115.9,112.3,113.9,113.9,2.9,2.0,2.0,0.187,1.54,17.1,0.008,0.115,0.064,11.1,11.0,0.604,0.636,0.258,0.264,104.50,103.37,86.14,103.37,0.122,3928,453,889,8.5,16.8,0.510,125,163,437,299,31,100,87,87,344,409,409,167,193,193,145,325,274,508,284,374,356,357,79,60,58,62,83,139,139,98,20,27,23,36,117
543,1630192,Zeke Nnaji,Zeke,1610612743,DEN,24.0,40,28,12,0.700,9.3,107.8,108.3,108.3,107.0,106.2,106.2,0.8,2.1,2.1,0.039,1.83,9.3,0.052,0.090,0.072,5.1,5.1,0.593,0.599,0.119,0.121,101.75,101.93,84.94,101.93,0.067,787,49,91,1.2,2.3,0.538,259,113,171,75,417,331,321,321,137,110,110,223,191,191,515,231,487,188,405,319,41,41,94,150,480,482,249,216,216,409,352,370,417,445,80
544,1630533,Ziaire Williams,Ziaire,1610612751,BKN,23.0,46,18,28,0.391,24.0,106.4,107.5,107.5,113.7,115.5,115.5,-7.4,-8.0,-8.0,0.093,1.38,12.3,0.042,0.168,0.102,8.9,8.9,0.491,0.536,0.181,0.185,98.84,97.57,81.31,97.57,0.083

In [ ]:
#lets start with individual player metrics. 
from sqlalchemy import create_engine, text
import pandas as pd
import math
from dotenv import load_dotenv
import os

load_dotenv()

def get_team_player_scores(teamName):
    #create our engine for creating sql entries - this wont work right now 
    DATABASE_URL = os.getenv("DATABASE_URL")

    engine = create_engine(DATABASE_URL)
    
    team_name = teamName
    
    #retreive players we want stats for 
    #take those players stats from the advanced table
    with engine.connect() as conn:
        with conn.begin():
            queries = {
                'hawks_roster_estimates': """
                	SELECT 
                    	t."TEAM_NAME",
                    	p.*,
                    	c."CLUTCH_SCORE_PCT",
                    	c."CLUTCH_USAGE_RATE",
                    	c."CLUTCH_NET_RATING",
                    	m."GP" as TOTAL_GAMES,
                    	m."MIN" as TOTAL_MINUTES,
                    	m."PTS" as AVG_POINTS,
                    	m."AST" as AVG_ASSISTS
                    FROM "all_teams_misc_stats" t
                    JOIN "all_players_estimated_stats" p
                    	ON t."TEAM_ID" = p."TEAM_ID"
                    LEFT JOIN "player_clutch_stats" c
                    	ON p."PLAYER_ID" = c."PLAYER_ID"
                    LEFT JOIN "all_players_misc_stats" m
                    	ON p."PLAYER_ID" = m."PLAYER_ID"
                    WHERE t."TEAM_NAME" = :team_name
    
                """,
                'hawks_roster_advanced': """
                	SELECT 
                    	t."TEAM_NAME",
                    	p.*,
                    	c."CLUTCH_SCORE_PCT",
                    	c."CLUTCH_USAGE_RATE",
                    	c."CLUTCH_NET_RATING",
                    	m."GP" as TOTAL_GAMES,
                    	m."MIN" as TOTAL_MINUTES
                    FROM "all_teams_misc_stats" t
                    JOIN "all_players_advanced_stats" p
                    	ON t."TEAM_ID" = p."TEAM_ID"
                    LEFT JOIN "player_clutch_stats" c
                    	ON p."PLAYER_ID" = c."PLAYER_ID"
                    LEFT JOIN "all_players_misc_stats" m
                    	ON p."PLAYER_ID" = m."PLAYER_ID"
                    WHERE t."TEAM_NAME" = :team_name
                """
            }
            #starters
            dfs = {name: pd.read_sql_query(text(query), engine, params={'team_name': team_name})
                  for name, query in queries.items()}
            #players with over 25 minutes of game time
            dfs['all_star_player'] = dfs['hawks_roster_advanced'][
                (dfs['hawks_roster_advanced']['MIN'] > 25)]
            
            #rotation players 
            dfs['key_rotation_player'] = dfs['hawks_roster_advanced'][
                (dfs['hawks_roster_advanced']['MIN'] > 15) & 
                (dfs['hawks_roster_advanced']['MIN'] < 25)]
    
    #get high impact stats - player impact rating, net rating, usg pct, plus/minus adjsuted for minutes
    #get efficiency stats - true shooting, turnover rate 
    #get availability - least important
    
    #stats for all star players
    dfs['all_star_estimates'] = pd.merge(
        dfs['hawks_roster_estimates'],
        dfs['all_star_player'][['PLAYER_ID','PLAYER_NAME', 'PIE', 'TS_PCT', 'MIN', 'NET_RATING', 'GP', 'CLUTCH_SCORE_PCT', 'CLUTCH_USAGE_RATE', 'CLUTCH_NET_RATING']]#these are the columns we are adding from all_star_player
    )[['PLAYER_ID', 'PLAYER_NAME', 'PIE', 'E_NET_RATING', 'E_USG_PCT', 'TS_PCT', 'E_TOV_PCT', 'MIN', 'NET_RATING', 'GP', 'CLUTCH_SCORE_PCT', 'CLUTCH_USAGE_RATE', 'CLUTCH_NET_RATING']] #these are the columns we are keeping from the final dataframe
    
    #stats for key rotation players
    dfs['key_rotation_estimates'] = pd.merge(
        dfs['hawks_roster_estimates'],
        dfs['key_rotation_player'][['PLAYER_ID', 'PLAYER_NAME', 'PIE', 'TS_PCT', 'MIN', 'NET_RATING', 'GP', 'CLUTCH_SCORE_PCT', 'CLUTCH_USAGE_RATE', 'CLUTCH_NET_RATING']]
    )[['PLAYER_ID', 'PLAYER_NAME', 'PIE', 'E_NET_RATING', 'E_USG_PCT', 'TS_PCT', 'E_TOV_PCT', 'MIN', 'NET_RATING', 'GP', 'CLUTCH_SCORE_PCT', 'CLUTCH_USAGE_RATE', 'CLUTCH_NET_RATING']]
    
    #apply weights to key points 
    #star players 1x weight, key rotation .75 weight 
    #bench .5 weight 
    
    star_players = dfs['all_star_estimates']
    key_rotation_players = dfs['key_rotation_estimates']

    # player_scores = calculate_player_metrics(star_players, key_rotation_players)
    # return player_scores
    
get_team_player_scores(teamName)